In [73]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style

# Preprocesado y modelado
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders
from sklearn.preprocessing import OrdinalEncoder
import missingno
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [74]:
import pandas as pd

# Cargamos el archivo csv con los datos

df_central = pd.read_csv("C:/Users/Windows 10/Desktop/MASTER DATASCIENCE/TFM/df_central_2_1.csv")

df_central.head()


,Unnamed: 0,Fecha,Period,PREVISION,E_SIMEL,DESVIO,f_PREV_HIGH,f_PREV_LOW,f_RUN,Dia_Semana,Es_fin_semana,Año,Mes,Día
0,0,2021-01-01,1,0.0,0.0,0.0,0,0,0,4,False,2021,1,1
1,1,2021-01-01,2,0.0,0.0,0.0,0,0,0,4,False,2021,1,1
2,2,2021-01-01,3,0.0,0.0,0.0,0,0,0,4,False,2021,1,1
3,3,2021-01-01,4,0.0,0.0,0.0,0,0,0,4,False,2021,1,1
4,4,2021-01-01,5,0.0,0.0,0.0,0,0,0,4,False,2021,1,1


In [75]:
# Eliminamos las columnas innecesarias

df_central = df_central.drop(columns=['Unnamed: 0', 'DESVIO', 'f_PREV_HIGH', 'f_PREV_LOW'])



In [76]:
# Convertimos la columna 'Fecha' a Datetime para hacer las división en dos dfs

df_central['Fecha'] = pd.to_datetime(df_central['Fecha'])

# Dividimos el DataFrame en dos según las fechas especificas

df_inicio = df_central[df_central['Fecha'] <= '2023-10-31']
df_final = df_central[df_central['Fecha'] >= '2023-11-05']

# Eliminamos la columna 'Fecha' de ambos DataFrames para poder preparar el modelo de Deep Learning

df_inicio = df_inicio.drop(columns=['Fecha'])
df_final = df_final.drop(columns=['Fecha'])



In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Preparamos df_inicio eliminando la variables objetivo del conjunto de entrenamiento (X) y especificamos la variable objetivo del conjunto de prueba (y)

X = df_inicio.drop('E_SIMEL', axis=1)
y = df_inicio['E_SIMEL']

# Dividimos los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizamos las características, paso necesario para el modelo de deep learning

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verificamos las dimensiones de los conjuntos de datos para asegurarnos de que todo está correcto

(X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape)


((19764, 8), (4942, 8), (19764,), (4942,))

In [78]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import regularizers

# Definimos la arquitectura del modelo, con dos capas ocultas de 64 unidades cada una con la función de activación 'relu', que proporciona
# capacidad de modelado no lineal.
# una capa de salida sin función de activación





model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],), kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=25, verbose=1, mode='min', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.001)

history = model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, batch_size=32, callbacks=[early_stopping, reduce_lr], verbose=1)


# Evaluamos el conjunto de prueba

test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Loss en el conjunto de prueba: {test_loss}, MAE: {test_mae}")


Epoch 1/100
495/495 [==============================] - 2s 2ms/step - loss: 89.3838 - mae: 4.8060 - val_loss: 41.5977 - val_mae: 3.2025 - lr: 0.0010
Epoch 2/100
495/495 [==============================] - 1s 2ms/step - loss: 55.8058 - mae: 3.8070 - val_loss: 42.2489 - val_mae: 3.2513 - lr: 0.0010
Epoch 3/100
495/495 [==============================] - 1s 2ms/step - loss: 51.4095 - mae: 3.6352 - val_loss: 38.7833 - val_mae: 2.9453 - lr: 0.0010
Epoch 4/100
495/495 [==============================] - 1s 1ms/step - loss: 51.8296 - mae: 3.6438 - val_loss: 38.5807 - val_mae: 2.8864 - lr: 0.0010
Epoch 5/100
495/495 [==============================] - 1s 2ms/step - loss: 52.1989 - mae: 3.6228 - val_loss: 38.4409 - val_mae: 2.8712 - lr: 0.0010
Epoch 6/100
495/495 [==============================] - 1s 2ms/step - loss: 50.2241 - mae: 3.5396 - val_loss: 37.7661 - val_mae: 2.8153 - lr: 0.0010
Epoch 7/100
495/495 [==============================] - 1s 2ms/step - loss: 49.3490 - mae: 3.4850 - val_loss: 38.

In [79]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split


In [80]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingRegressor

# Configuramos el imputador MICE con GradientBoostingRegressor y el estimador

mice_imputer = IterativeImputer(estimator=GradientBoostingRegressor(
                                    n_estimators=100,
                                    max_depth=10,
                                    min_samples_split=4,
                                    min_samples_leaf=2,
                                    max_features='sqrt'),
                                    max_iter=10, random_state=42)

# Preparamos los datos para el entrenamiento del imputador MICE eliminando la variable objetivo E_SIMEL

X_mice = df_inicio.drop(columns=['E_SIMEL'])

# Entrenamos el imputador MICE

mice_imputer.fit(X_mice)

# Imprimimos confirmación

"Imputador MICE entrenado con éxito."



'Imputador MICE entrenado con éxito.'

In [81]:
# Seleccionamos las filas del dia 5 del df_final para hacer el tratamiento de la variable f_RUN

df_final_05_11 = df_final[(df_final['Año'] == 2023) & (df_final['Mes'] == 11) & (df_final['Día'] == 5)]

df_final_05_11_para_imputar = df_final_05_11.drop(['E_SIMEL'], axis=1)

df_final_05_11_para_imputar[['f_RUN']] = np.nan  # Primero convertimos la columna f_RUN a NaN

# En la variable creada para la imputación, predecimos los valores para las columnas siguientes

valores_imputados = mice_imputer.transform(df_final_05_11_para_imputar[['Period', 'PREVISION', 'f_RUN', 'Dia_Semana', 'Es_fin_semana', 'Año', 'Mes', 'Día']])

# Realmente solo queremos imputar la columna f_RUN, por lo tanto cogemos solo los valores imputados a la columna en concreto y
# establecemos una condición que si los valores son más grandes que 0.2 establecemos un 1, y si son inferiores establecemos un 0.

valores_imputados_f_RUN = np.where(valores_imputados[:, 2]> 0.2, 1, 0)

df_final_05_11.loc[:, 'f_RUN'] = valores_imputados_f_RUN



In [82]:
# Preparamos los datos de df_final_05_11 para la predicción quitando la variable objetivo

X_final_05_11 = df_final_05_11.drop(['E_SIMEL'], axis=1)

# Normalización de los datos

X_final_05_11_scaled = scaler.transform(X_final_05_11)

# Realizamos la predicciones de E_SIMEL con el modelo de deep learning

e_simel_predicciones = model.predict(X_final_05_11_scaled)

e_simel_predicciones = np.maximum(e_simel_predicciones, 0)

# Mostramos los resultados de la predicción

e_simel_predicciones[:25]



1/1 [==============================] - 0s 82ms/step


array([[0.        ],
       [0.21490622],
       [0.34728146],
       [0.33934116],
       [0.3314004 ],
       [0.30388498],
       [0.2760558 ],
       [0.2592516 ],
       [0.2417326 ],
       [0.22439814],
       [0.21672678],
       [0.21275568],
       [0.22775507],
       [0.27858782],
       [9.247498  ],
       [8.859642  ],
       [8.496704  ],
       [8.136265  ],
       [7.708955  ],
       [7.202019  ],
       [5.3416247 ],
       [0.7198763 ],
       [0.7693553 ],
       [0.82098913]], dtype=float32)

In [83]:
# Convertimos el array a una lista para facilitar la asignación a una nueva columna que llamamos 'Prediccion_E_SIMEL'

predicciones_lista = e_simel_predicciones.flatten().tolist()

# Asignar las predicciones a df_final_05_11

df_final_05_11['Prediccion_E_SIMEL'] = predicciones_lista

# Mostramos las primeras filas para verificar

df_final_05_11[['E_SIMEL', 'PREVISION', 'Prediccion_E_SIMEL']].head(25)




C:\Users\Windows 10\AppData\Local\Temp\ipykernel_72268\496075226.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_05_11['Prediccion_E_SIMEL'] = predicciones_lista


,E_SIMEL,PREVISION,Prediccion_E_SIMEL
24802,0.000,0.0,0.000000
24803,0.000,0.0,0.214906
24804,0.000,0.0,0.347281
24805,0.000,0.0,0.339341
24806,0.000,0.0,0.331400
24807,0.000,0.0,0.303885
24808,0.000,0.0,0.276056
24809,0.000,0.0,0.259252
24810,0.000,0.0,0.241733
24811,0.000,0.0,0.224398


In [84]:
# al igual que en todos los casos anteriores, hacemos un sumatorio y porcentaje de desviación para tener una primera idea de como van las predicciones

suma_real_05 = df_final_05_11['E_SIMEL'].sum()
suma_predicha_05 = df_final_05_11['Prediccion_E_SIMEL'].sum()
suma_prevision_05 = df_final_05_11['PREVISION'].sum()


if suma_real_05 != 0:
    desviacion_porcentual = 100 * (suma_predicha_05 - suma_real_05) / suma_real_05
else:
    desviacion_porcentual = float('inf')  # en caso de división por cero, retorna un valor especial para que no nos dé error



if suma_real_05 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_05 - suma_real_05) / suma_real_05
else:
    desviacion_porcentual_prevision = float('inf')  # en caso de división por cero, retorna un valor especial para que no nos dé error

print("Suma real: ", suma_real_05)
print("Suma predicha: ", suma_predicha_05)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_05)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  15.937000000000001
Suma predicha:  60.777005672454834
Desviación porcentual:  281.35788211366526 %
Suma previsión:  0.0
Desviación porcentual:  -100.0 %


In [85]:
# Una vez tenemos la primera predicción, actualizamos df_inicio con los datos del día 5. Así estamos simulando como sería
# el proceso de predicción en tiempo real

datos_dia_5 = df_final[(df_final['Año'] == 2023) & (df_final['Mes'] == 11) & (df_final['Día'] == 5)]
df_inicio_actualizado = pd.concat([df_inicio, datos_dia_5])

# Preparamos df_actualizado con los datos de df_inicio y los datos del día 5 para reentrenar el modelo de deep learning

X_actualizado = df_inicio_actualizado.drop('E_SIMEL', axis=1)
y_actualizado = df_inicio_actualizado['E_SIMEL']


# Normalizamos con fit_transform

X_total_scaled = scaler.fit_transform(X_actualizado)  # Utilizamos fit_transform 

# Y reentrenamos el modelo de deep learning con todos los datos con los parámetros utilizados para el entrenamiento inicial

model.fit(X_total_scaled, y_actualizado, epochs=100, validation_split=0.2, batch_size=32, callbacks=[early_stopping, reduce_lr], verbose=1)

# reentrenamos también el modelo imputador con los nuevos datos con todas las variables menos con E_SIMEL

mice_imputer.fit(df_inicio_actualizado[['Period', 'PREVISION', 'f_RUN', 'Dia_Semana', 'Es_fin_semana', 'Año', 'Mes', 'Día']])



Epoch 1/100
619/619 [==============================] - 1s 2ms/step - loss: 37.7662 - mae: 3.2514 - val_loss: 41.4291 - val_mae: 3.3071 - lr: 0.0010
Epoch 2/100
619/619 [==============================] - 1s 1ms/step - loss: 37.3600 - mae: 3.2263 - val_loss: 47.4276 - val_mae: 3.8756 - lr: 0.0010
Epoch 3/100
619/619 [==============================] - 1s 1ms/step - loss: 37.4137 - mae: 3.2307 - val_loss: 44.2947 - val_mae: 3.5394 - lr: 0.0010
Epoch 4/100
619/619 [==============================] - 1s 1ms/step - loss: 37.5549 - mae: 3.2209 - val_loss: 48.8770 - val_mae: 3.7361 - lr: 0.0010
Epoch 5/100
619/619 [==============================] - 1s 1ms/step - loss: 36.9402 - mae: 3.2086 - val_loss: 47.0773 - val_mae: 3.6067 - lr: 0.0010
Epoch 6/100
619/619 [==============================] - 1s 1ms/step - loss: 36.9625 - mae: 3.2151 - val_loss: 55.3712 - val_mae: 4.0188 - lr: 0.0010
Epoch 7/100
619/619 [==============================] - 1s 1ms/step - loss: 36.3267 - mae: 3.1966 - val_loss: 52.

IterativeImputer(estimator=GradientBoostingRegressor(max_depth=10,
                                                     max_features='sqrt',
                                                     min_samples_leaf=2,
                                                     min_samples_split=4),
                 random_state=42)

In [86]:
import numpy as np

# Seleccionamos las filas del dia 6 del df_final para hacer el tratamiento de la variable f_RUN

df_final_06_11 = df_final[(df_final['Año'] == 2023) & (df_final['Mes'] == 11) & (df_final['Día'] == 6)]

df_final_06_11_para_imputar = df_final_06_11.drop(['E_SIMEL'], axis=1)

df_final_06_11_para_imputar[['f_RUN']] = np.nan  # Primero convertimos la columna f_RUN a NaN

# En la variable creada para la imputación, predecimos los valores para las columnas siguientes

valores_imputados = mice_imputer.transform(df_final_06_11_para_imputar[['Period', 'PREVISION', 'f_RUN', 'Dia_Semana', 'Es_fin_semana', 'Año', 'Mes', 'Día']])

# Realmente solo queremos imputar la columna f_RUN, por lo tanto cogemos solo los valores imputados a la columna en concreto

valores_imputados_f_RUN = np.where(valores_imputados[:, 2]> 0.2, 1, 0)

df_final_06_11.loc[:, 'f_RUN'] = valores_imputados_f_RUN


# Verificamos que los valores han sido imputados correctamente
# df_final_05_11.head(25)

# Prepararamos los datos de df_final_06_11 para la predicción

X_final_06_11 = df_final_06_11.drop(['E_SIMEL'], axis=1)

# Normalizamos los datos de X_final_05_11 

X_final_06_11_scaled = scaler.transform(X_final_06_11)

# Realizamos las predicciones de E_SIMEL

e_simel_predicciones_06 = model.predict(X_final_06_11_scaled)

# Para que no haya valores negativos los transformamos a cero

e_simel_predicciones_06 = np.maximum(e_simel_predicciones_06, 0)

# Mostrar las primeras 5 predicciones
# e_simel_predicciones[:25]

# Convertimos el array de predicciones a una lista para facilitar la asignación

predicciones_lista = e_simel_predicciones_06.flatten().tolist()

# Asignamos las predicciones a una nueva columna en el Datadrame df_final_06_11

df_final_06_11['Prediccion_E_SIMEL'] = predicciones_lista

# Visualizamos las columnas para ver el resultado

df_final_06_11[['E_SIMEL', 'PREVISION', 'Prediccion_E_SIMEL']].head(25)



1/1 [==============================] - 0s 15ms/step


C:\Users\Windows 10\AppData\Local\Temp\ipykernel_72268\3222644168.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_06_11['Prediccion_E_SIMEL'] = predicciones_lista


,E_SIMEL,PREVISION,Prediccion_E_SIMEL
24826,0.000,0.0,0.000000
24827,0.000,0.0,0.204602
24828,0.000,0.0,0.226279
24829,0.000,0.0,0.192582
24830,0.000,0.0,0.160977
24831,0.000,0.0,0.138354
24832,0.000,0.0,0.115732
24833,0.000,0.0,0.093109
24834,0.000,0.0,0.076867
24835,0.000,0.0,0.050153


In [87]:
# Sumatorios y porcentajes de desviación

suma_real_06 = df_final_06_11['E_SIMEL'].sum()
suma_predicha_06 = df_final_06_11['Prediccion_E_SIMEL'].sum()
suma_prevision_06 = df_final_06_11['PREVISION'].sum()


if suma_real_06 != 0:
    desviacion_porcentual = 100 * (suma_predicha_06 - suma_real_06) / suma_real_06
else:
    desviacion_porcentual = float('inf')  # en caso de división por cero, retorna un valor especial para que no nos dé error



if suma_real_06 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_06 - suma_real_06) / suma_real_06
else:
    desviacion_porcentual_prevision = float('inf')  # en caso de división por cero, retorna un valor especial para que no nos dé error

print("Suma real: ", suma_real_06)
print("Suma predicha: ", suma_predicha_06)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_06)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  58.542
Suma predicha:  111.9960675239563
Desviación porcentual:  91.3089192783921 %
Suma previsión:  159.4
Desviación porcentual:  172.2831471422227 %


In [88]:
# Creamos un función para agilizar el proceso de actualización, reentreno de los modelos, imputación, predicción y cálculo de las métricas

def predecir_y_actualizar_para_un_dia(dia_actual, dia_siguiente,  mes, año, df_inicio_actualizado, df_final, modelo_deep, imputador):
    """
    Función para actualizar el conjunto de entrenamiento con los datos reales de un día específico,
    realizar la imputación para el día siguiente y predecir los valores de E_SIMEL para ese día.

    Args:
    dia_actual (int): Día actual para el que se actualizarán los datos.
    dia_siguiente (int): Datos del día que queremos hacer las imputaciones y la predicción
    mes (int): Mes del día actual.
    ano (int): Año del día actual.
    df_inicio_actualizado (DataFrame): DataFrame actualizado con los datos hasta el día anterior.
    df_final (DataFrame): DataFrame con los datos a predecir.
    modelo_deep (DeepLearning de TencerFlow): Modelo de Deep Learning entrenado.
    imputador (IterativeImputer): Imputador MICE entrenado.

    Returns:
    DataFrame: DataFrame con las predicciones para el día siguiente.
    DataFrame: DataFrame actualizado con los datos reales del día actual.
    """
    # Actualización de df_actualizado con los datos de dia_actual

    datos_dia_actual = df_final[(df_final['Año'] == año) & (df_final['Mes'] == mes) & (df_final['Día'] == dia_actual)]
    df_inicio_actualizado = pd.concat([df_inicio_actualizado, datos_dia_actual])

    
    # Prepararamos los datos para el modelo de deep learning

    X_actualizado = df_inicio_actualizado.drop('E_SIMEL', axis=1)
    y_actualizado = df_inicio_actualizado['E_SIMEL']

        
    # Normalizaación las características

    X_actualizado_scaled = scaler.transform(X_actualizado)

    early_stopping = EarlyStopping(monitor='val_loss', patience=25, verbose=1, mode='min', restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.001)

    # Reentrenamos el modelo de deep learning con todos los datos pasándole los parámetros de modelado

    modelo_deep.fit(X_actualizado_scaled, y_actualizado, epochs=100, validation_split=0.2, batch_size=32, callbacks=[early_stopping, reduce_lr], verbose=1)

    # Reentrenamos el imputador para todas las variables menos con E_SIMEL

    imputador.fit(df_inicio_actualizado[['Period', 'PREVISION', 'f_RUN', 'Dia_Semana', 'Es_fin_semana', 'Año', 'Mes', 'Día']])    

    

      # Imputación de valores a la columna f_RUN para la predicción

    df_dia_siguiente = df_final[(df_final['Año'] == año) & (df_final['Mes'] == mes) & (df_final['Día'] == dia_siguiente)]
    df_dia_siguiente_para_imputar = df_dia_siguiente.drop(['E_SIMEL'], axis=1)
    df_dia_siguiente_para_imputar[['f_RUN']] = np.nan  
    
    valores_imputados = imputador.transform(df_dia_siguiente_para_imputar)
    
    df_dia_siguiente.loc[:, 'f_RUN'] = np.where(valores_imputados[:, 2] > 0.2, 1, 0) 

    
    
    # Preparamos los datos para la predicción con el modelo de deep learning

    X_prediccion = df_dia_siguiente.drop(['E_SIMEL'], axis=1)

    # Normalizamos los datos 

    X_prediccion_scaled = scaler.transform(X_prediccion)

    # Realizamos las predicciones de E_SIMEL
    predicted_e_simel = model.predict(X_prediccion_scaled)

    # Transformamos los valores negativos de la predicción a cero

    predicted_e_simel = np.maximum(predicted_e_simel, 0)

    # Mostramos las predicciones

    # e_simel_predicciones[:25]

    # Convertimos el array de las predicciones a una lista para facilitar la asignación
    predicciones_lista = predicted_e_simel.flatten().tolist()

    df_predicciones = df_dia_siguiente[['Año', 'Mes', 'Día', 'PREVISION', 'E_SIMEL']].copy()

    # En el df_prediciones creamos una nueva columna con las predicciones

    df_predicciones['Prediccion_E_SIMEL'] = predicciones_lista

    # Mostramos el resultado

    df_predicciones[['E_SIMEL', 'PREVISION', 'Prediccion_E_SIMEL']].head(25)


    

    # Calculamos las métricas: mse (Error cuadrático medio), r2 (coeficiente de determinación) y el mae(error medio absoluto)

    mse = mean_squared_error(df_predicciones['E_SIMEL'], df_predicciones['Prediccion_E_SIMEL'])
    r2 = r2_score(df_predicciones['E_SIMEL'], df_predicciones['Prediccion_E_SIMEL'])
    mae = mean_absolute_error(df_predicciones['E_SIMEL'], df_predicciones['Prediccion_E_SIMEL'])


    return df_predicciones, df_inicio_actualizado, mse, r2, mae

In [89]:
# Llamamos a la función

dia_actual = 6    # Actualización de datos con los que reentrenamos los modelos
dia_siguiente = 7    # Preparación de datos para la imputación y predicción
df_predicciones_07_11, df_inicio_actualizado, mse_07_11, r2_07_11, mae_07_11 = predecir_y_actualizar_para_un_dia(dia_actual, dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_07_11, "R²:", r2_07_11, "MAE:", mae_07_11)

Epoch 1/100
619/619 [==============================] - 1s 2ms/step - loss: 37.2893 - mae: 3.2148 - val_loss: 45.1692 - val_mae: 3.5512 - lr: 0.0010
Epoch 2/100
619/619 [==============================] - 1s 1ms/step - loss: 37.0058 - mae: 3.2114 - val_loss: 49.4829 - val_mae: 3.7362 - lr: 0.0010
Epoch 3/100
619/619 [==============================] - 1s 1ms/step - loss: 37.1679 - mae: 3.2269 - val_loss: 46.6995 - val_mae: 3.5558 - lr: 0.0010
Epoch 4/100
619/619 [==============================] - 1s 1ms/step - loss: 37.2355 - mae: 3.2277 - val_loss: 54.4792 - val_mae: 3.8986 - lr: 0.0010
Epoch 5/100
619/619 [==============================] - 1s 1ms/step - loss: 36.6911 - mae: 3.2239 - val_loss: 57.7612 - val_mae: 4.1846 - lr: 0.0010
Epoch 6/100
619/619 [==============================] - 1s 1ms/step - loss: 36.7420 - mae: 3.1964 - val_loss: 71.0054 - val_mae: 4.7679 - lr: 0.0010
Epoch 7/100
619/619 [==============================] - 1s 1ms/step - loss: 37.7519 - mae: 3.2320 - val_loss: 60.

In [90]:
# Y como en la predicción anterior calculamos los sumatorios y los porcentajes de desviación

suma_real_07_11 = df_predicciones_07_11['E_SIMEL'].sum()
suma_predicha_07_11 = df_predicciones_07_11['Prediccion_E_SIMEL'].sum()
suma_prevision_07_11 = df_predicciones_07_11['PREVISION'].sum()


if suma_real_07_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_07_11 - suma_real_07_11) / suma_real_07_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_07_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_07_11 - suma_real_07_11) / suma_real_07_11
else:
    desviacion_porcentual_prevision = float('inf')  
    

print("Suma real: ", suma_real_07_11)
print("Suma predicha: ", suma_predicha_07_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_07_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  67.97999999999999
Suma predicha:  74.83839845657349
Desviación porcentual:  10.088847391252571 %
Suma previsión:  24.0
Desviación porcentual:  -64.6954986760812 %


In [91]:
# Llamamos a la función. Actualización de datos reales del día 7 para reentrenar los modelos
# Preparación de los datos del día posterior, en este caso del día 8, para la imputación y predicción

dia_actual = 7
dia_siguiente = 8

df_predicciones_08_11, df_inicio_actualizado, mse_08_11, r2_08_11, mae_08_11 = predecir_y_actualizar_para_un_dia(dia_actual, dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_08_11, "R²:", r2_08_11, "MAE:", mae_08_11)

Epoch 1/100
620/620 [==============================] - 1s 2ms/step - loss: 37.0623 - mae: 3.2158 - val_loss: 44.6343 - val_mae: 3.4928 - lr: 0.0010
Epoch 2/100
620/620 [==============================] - 1s 1ms/step - loss: 37.0104 - mae: 3.2055 - val_loss: 55.5635 - val_mae: 4.2017 - lr: 0.0010
Epoch 3/100
620/620 [==============================] - 1s 1ms/step - loss: 36.6546 - mae: 3.2052 - val_loss: 48.4874 - val_mae: 3.7553 - lr: 0.0010
Epoch 4/100
620/620 [==============================] - 1s 1ms/step - loss: 37.8237 - mae: 3.2601 - val_loss: 49.5633 - val_mae: 3.8691 - lr: 0.0010
Epoch 5/100
620/620 [==============================] - 1s 1ms/step - loss: 36.6038 - mae: 3.1920 - val_loss: 53.4723 - val_mae: 4.0307 - lr: 0.0010
Epoch 6/100
620/620 [==============================] - 1s 1ms/step - loss: 36.4080 - mae: 3.1915 - val_loss: 60.0158 - val_mae: 4.3632 - lr: 0.0010
Epoch 7/100
620/620 [==============================] - 1s 2ms/step - loss: 36.9793 - mae: 3.2165 - val_loss: 58.

In [92]:
# Sumatorios y porcentajes

suma_real_08_11 = df_predicciones_08_11['E_SIMEL'].sum()
suma_predicha_08_11 = df_predicciones_08_11['Prediccion_E_SIMEL'].sum()
suma_prevision_08_11 = df_predicciones_08_11['PREVISION'].sum()


if suma_real_08_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_08_11 - suma_real_08_11) / suma_real_08_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_08_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_08_11 - suma_real_08_11) / suma_real_08_11
else:
    desviacion_porcentual_prevision = float('inf')  
    

print("Suma real: ", suma_real_08_11)
print("Suma predicha: ", suma_predicha_08_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_08_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  11.296999999999999
Suma predicha:  68.19218826293945
Desviación porcentual:  503.6309485964368 %
Suma previsión:  0.0
Desviación porcentual:  -100.0 %


In [93]:
# Seguimos con el mismo proceso anterior. Llamamos a la función con los días específicos que queremos actualizar, imputar y predecir.

dia_actual = 8
dia_siguiente = 9

df_predicciones_09_11, df_inicio_actualizado, mse_09_11, r2_09_11, mae_09_11 = predecir_y_actualizar_para_un_dia(dia_actual, dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_09_11, "R²:", r2_09_11, "MAE:", mae_09_11)

Epoch 1/100
621/621 [==============================] - 1s 2ms/step - loss: 36.6965 - mae: 3.1984 - val_loss: 51.1555 - val_mae: 3.9443 - lr: 0.0010
Epoch 2/100
621/621 [==============================] - 1s 2ms/step - loss: 37.0854 - mae: 3.2236 - val_loss: 51.4634 - val_mae: 3.8962 - lr: 0.0010
Epoch 3/100
621/621 [==============================] - 1s 1ms/step - loss: 36.7944 - mae: 3.2153 - val_loss: 50.3702 - val_mae: 3.7339 - lr: 0.0010
Epoch 4/100
621/621 [==============================] - 1s 1ms/step - loss: 36.9650 - mae: 3.2309 - val_loss: 51.1323 - val_mae: 3.7002 - lr: 0.0010
Epoch 5/100
621/621 [==============================] - 1s 1ms/step - loss: 36.4193 - mae: 3.1897 - val_loss: 53.5188 - val_mae: 3.8856 - lr: 0.0010
Epoch 6/100
621/621 [==============================] - 1s 1ms/step - loss: 36.3971 - mae: 3.2020 - val_loss: 63.0759 - val_mae: 4.2346 - lr: 0.0010
Epoch 7/100
621/621 [==============================] - 1s 1ms/step - loss: 37.0889 - mae: 3.2388 - val_loss: 59.

In [94]:
# Sumatorios y procentajes

suma_real_09_11 = df_predicciones_09_11['E_SIMEL'].sum()
suma_predicha_09_11 = df_predicciones_09_11['Prediccion_E_SIMEL'].sum()
suma_prevision_09_11 = df_predicciones_09_11['PREVISION'].sum()


if suma_real_09_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_09_11 - suma_real_09_11) / suma_real_09_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_09_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_09_11 - suma_real_09_11) / suma_real_09_11
else:
    desviacion_porcentual_prevision = float('inf')  
    

print("Suma real: ", suma_real_09_11)
print("Suma predicha: ", suma_predicha_09_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_09_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  102.43700000000001
Suma predicha:  104.05211687088013
Desviación porcentual:  1.5766928657419828 %
Suma previsión:  127.30000000000001
Desviación porcentual:  24.271503460663627 %


In [95]:
# Llamamos a la función
dia_actual = 9
dia_siguiente = 10

df_predicciones_10_11, df_inicio_actualizado, mse_10_11, r2_10_11, mae_10_11 = predecir_y_actualizar_para_un_dia(dia_actual,dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_10_11, "R²:", r2_10_11, "MAE:", mae_10_11)

Epoch 1/100
621/621 [==============================] - 1s 2ms/step - loss: 36.2771 - mae: 3.1823 - val_loss: 50.8057 - val_mae: 3.7420 - lr: 0.0010
Epoch 2/100
621/621 [==============================] - 1s 1ms/step - loss: 37.5122 - mae: 3.2454 - val_loss: 55.7127 - val_mae: 3.9159 - lr: 0.0010
Epoch 3/100
621/621 [==============================] - 1s 1ms/step - loss: 36.9573 - mae: 3.2174 - val_loss: 58.8828 - val_mae: 4.1932 - lr: 0.0010
Epoch 4/100
621/621 [==============================] - 1s 1ms/step - loss: 36.9703 - mae: 3.2190 - val_loss: 55.6861 - val_mae: 3.9032 - lr: 0.0010
Epoch 5/100
621/621 [==============================] - 1s 1ms/step - loss: 36.8112 - mae: 3.2120 - val_loss: 54.9308 - val_mae: 3.9161 - lr: 0.0010
Epoch 6/100
621/621 [==============================] - 1s 1ms/step - loss: 36.8433 - mae: 3.2471 - val_loss: 56.0519 - val_mae: 4.0305 - lr: 0.0010
Epoch 7/100
621/621 [==============================] - 1s 1ms/step - loss: 36.9512 - mae: 3.2344 - val_loss: 58.

In [96]:
# Sumatorios y porcentajes

suma_real_10_11 = df_predicciones_10_11['E_SIMEL'].sum()
suma_predicha_10_11 = df_predicciones_10_11['Prediccion_E_SIMEL'].sum()
suma_prevision_10_11 = df_predicciones_10_11['PREVISION'].sum()


if suma_real_10_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_10_11 - suma_real_10_11) / suma_real_10_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_10_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_10_11 - suma_real_10_11) / suma_real_10_11
else:
    desviacion_porcentual_prevision = float('inf')  

print("Suma real: ", suma_real_10_11)
print("Suma predicha: ", suma_predicha_10_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_10_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  58.955
Suma predicha:  133.0757875442505
Desviación porcentual:  125.72434491434228 %
Suma previsión:  112.6
Desviación porcentual:  90.99313035365958 %


In [97]:
# Llamamos a la función

dia_actual = 10
dia_siguiente = 13

df_predicciones_13_11, df_inicio_actualizado, mse_13_11, r2_13_11, mae_13_11 = predecir_y_actualizar_para_un_dia(dia_actual,dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_13_11, "R²:", r2_13_11, "MAE:", mae_13_11)

Epoch 1/100
622/622 [==============================] - 1s 2ms/step - loss: 37.1600 - mae: 3.2335 - val_loss: 54.4703 - val_mae: 3.8692 - lr: 0.0010
Epoch 2/100
622/622 [==============================] - 1s 2ms/step - loss: 36.1284 - mae: 3.2012 - val_loss: 52.0060 - val_mae: 3.7854 - lr: 0.0010
Epoch 3/100
622/622 [==============================] - 1s 1ms/step - loss: 36.4517 - mae: 3.2284 - val_loss: 71.9993 - val_mae: 4.5333 - lr: 0.0010
Epoch 4/100
622/622 [==============================] - 1s 1ms/step - loss: 36.2203 - mae: 3.2174 - val_loss: 72.1539 - val_mae: 4.5940 - lr: 0.0010
Epoch 5/100
622/622 [==============================] - 1s 1ms/step - loss: 36.3564 - mae: 3.2303 - val_loss: 65.0311 - val_mae: 4.3789 - lr: 0.0010
Epoch 6/100
622/622 [==============================] - 1s 1ms/step - loss: 36.3498 - mae: 3.2087 - val_loss: 63.9549 - val_mae: 4.3214 - lr: 0.0010
Epoch 7/100
622/622 [==============================] - 1s 1ms/step - loss: 36.1364 - mae: 3.2101 - val_loss: 67.

In [98]:
# Sumas y porcentajes

suma_real_13_11 = df_predicciones_13_11['E_SIMEL'].sum()
suma_predicha_13_11 = df_predicciones_13_11['Prediccion_E_SIMEL'].sum()
suma_prevision_13_11 = df_predicciones_13_11['PREVISION'].sum()

if suma_real_13_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_13_11 - suma_real_13_11) / suma_real_13_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_13_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_13_11 - suma_real_13_11) / suma_real_13_11
else:
    desviacion_porcentual_prevision = float('inf')  

print("Suma real: ", suma_real_13_11)
print("Suma predicha: ", suma_predicha_13_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_13_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  107.141
Suma predicha:  44.470181941986084
Desviación porcentual:  -58.49377741295481 %
Suma previsión:  20.900000000000002
Desviación porcentual:  -80.492995211917 %


In [99]:
# Llamamos a la función

dia_actual = 13
dia_siguiente = 14

df_predicciones_14_11, df_inicio_actualizado, mse_14_11, r2_14_11, mae_14_11 = predecir_y_actualizar_para_un_dia(dia_actual,dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_14_11, "R²:", r2_14_11, "MAE:", mae_14_11)

Epoch 1/100
622/622 [==============================] - 1s 2ms/step - loss: 36.8643 - mae: 3.2293 - val_loss: 57.9758 - val_mae: 4.0666 - lr: 0.0010
Epoch 2/100
622/622 [==============================] - 1s 1ms/step - loss: 36.4722 - mae: 3.2059 - val_loss: 73.0620 - val_mae: 4.6849 - lr: 0.0010
Epoch 3/100
622/622 [==============================] - 1s 1ms/step - loss: 36.4243 - mae: 3.2086 - val_loss: 64.1403 - val_mae: 4.1956 - lr: 0.0010
Epoch 4/100
622/622 [==============================] - 1s 1ms/step - loss: 36.2233 - mae: 3.2114 - val_loss: 58.3026 - val_mae: 4.0104 - lr: 0.0010
Epoch 5/100
622/622 [==============================] - 1s 1ms/step - loss: 35.7910 - mae: 3.1804 - val_loss: 75.0239 - val_mae: 4.5983 - lr: 0.0010
Epoch 6/100
622/622 [==============================] - 1s 1ms/step - loss: 36.1827 - mae: 3.2289 - val_loss: 61.2207 - val_mae: 4.1000 - lr: 0.0010
Epoch 7/100
622/622 [==============================] - 1s 2ms/step - loss: 35.6655 - mae: 3.1990 - val_loss: 78.

In [100]:
# Sumatorios y porcentajes

suma_real_14_11 = df_predicciones_14_11['E_SIMEL'].sum()
suma_predicha_14_11 = df_predicciones_14_11['Prediccion_E_SIMEL'].sum()
suma_prevision_14_11 = df_predicciones_14_11['PREVISION'].sum()

if suma_real_14_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_14_11 - suma_real_14_11) / suma_real_14_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_14_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_14_11 - suma_real_14_11) / suma_real_14_11
else:
    desviacion_porcentual_prevision = float('inf')  

print("Suma real: ", suma_real_14_11)
print("Suma predicha: ", suma_predicha_14_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_14_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  112.33099999999999
Suma predicha:  75.91315746307373
Desviación porcentual:  -32.42011780979984 %
Suma previsión:  120.5
Desviación porcentual:  7.272257880727503 %


In [101]:
# Llamamos a la función

dia_actual = 14
dia_siguiente = 15

df_predicciones_15_11, df_inicio_actualizado, mse_15_11, r2_15_11, mae_15_11 = predecir_y_actualizar_para_un_dia(dia_actual,dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_15_11, "R²:", r2_15_11, "MAE:", mae_15_11)

Epoch 1/100
623/623 [==============================] - 1s 2ms/step - loss: 36.6687 - mae: 3.2179 - val_loss: 55.5017 - val_mae: 4.0614 - lr: 0.0010
Epoch 2/100
623/623 [==============================] - 1s 1ms/step - loss: 37.5682 - mae: 3.2392 - val_loss: 67.4780 - val_mae: 4.4758 - lr: 0.0010
Epoch 3/100
623/623 [==============================] - 1s 1ms/step - loss: 36.2791 - mae: 3.2195 - val_loss: 71.8713 - val_mae: 4.5576 - lr: 0.0010
Epoch 4/100
623/623 [==============================] - 1s 1ms/step - loss: 36.0242 - mae: 3.2108 - val_loss: 66.7812 - val_mae: 4.3626 - lr: 0.0010
Epoch 5/100
623/623 [==============================] - 1s 2ms/step - loss: 36.3500 - mae: 3.2168 - val_loss: 65.6413 - val_mae: 4.3733 - lr: 0.0010
Epoch 6/100
623/623 [==============================] - 1s 2ms/step - loss: 36.7552 - mae: 3.2247 - val_loss: 78.7832 - val_mae: 4.8971 - lr: 0.0010
Epoch 7/100
623/623 [==============================] - 1s 2ms/step - loss: 37.1335 - mae: 3.2572 - val_loss: 62.

In [102]:
# Sumas y porcentajes

suma_real_15_11 = df_predicciones_15_11['E_SIMEL'].sum()
suma_predicha_15_11 = df_predicciones_15_11['Prediccion_E_SIMEL'].sum()
suma_prevision_15_11 = df_predicciones_15_11['PREVISION'].sum()

if suma_real_15_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_15_11 - suma_real_15_11) / suma_real_15_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_15_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_15_11 - suma_real_15_11) / suma_real_15_11
else:
    desviacion_porcentual_prevision = float('inf')  

print("Suma real: ", suma_real_15_11)
print("Suma predicha: ", suma_predicha_15_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_15_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  88.333
Suma predicha:  112.58826112747192
Desviación porcentual:  27.45888980049577 %
Suma previsión:  131.89999999999998
Desviación porcentual:  49.32131819365354 %


In [103]:
# Concatenamos todos los Dataframes que contienen las prediccions, previsiones y datos reales para calcular las méstricas en conjunto

df_predicciones_totales = pd.concat([df_final_05_11, df_final_06_11, df_predicciones_07_11, 
                                     df_predicciones_08_11, df_predicciones_09_11,df_predicciones_10_11, 
                                     df_predicciones_13_11, df_predicciones_14_11, df_predicciones_15_11])

In [104]:
# Cálculo de las métricas para ver si nos indican si mejoran los errores entren la predicción (Prediccion_E_SIMEL), previsión (PREVISION)y producción real(E_SIMEL)

def calcular_metricas(df):
    mae = mean_absolute_error(df['E_SIMEL'], df['Prediccion_E_SIMEL'])
    mse = mean_squared_error(df['E_SIMEL'], df['Prediccion_E_SIMEL'])
    r2 = r2_score(df['E_SIMEL'], df['Prediccion_E_SIMEL'])
    return mae, mse, r2

# Métricas para la predicciones

mae_pred, mse_pred, r2_pred = calcular_metricas(df_predicciones_totales)

# Cambiamos la columna de predicción por la de previsión

df_previsiones = df_predicciones_totales.copy()
df_previsiones['Prediccion_E_SIMEL'] = df_previsiones['PREVISION']

# Métricas para la previsión

mae_prev, mse_prev, r2_prev = calcular_metricas(df_previsiones)

# Visualizamos los resultados

print("MAE Predicciones: ", mae_pred)
print("MSE Predicciones: ", mse_pred)
print("R² Predicciones: ", r2_pred)
print("MAE Previsiones: ", mae_prev)
print("MSE Previsiones: ", mse_prev)
print("R² Previsiones: ", r2_prev)

MAE Predicciones:  3.021907988795528
MSE Predicciones:  33.06885109728008
R² Predicciones:  0.34617626181390015
MAE Previsiones:  2.3999490740740743
MSE Previsiones:  35.93822944907407
R² Previsiones:  0.2894440918718205


In [105]:
"""
Las métricas visualizadas nos indican que el modelo utilizado mejoran las métricas de las previsiones, vamos 
a ver a continuación si las métricas reflejan esta mejora en predicción respecto la previsión.

"""

'\nLas métricas visualizadas nos indican que el modelo utilizado mejoran las métricas de las previsiones, vamos \na ver a continuación si las métricas reflejan esta mejora en predicción respecto la previsión.\n\n'

In [106]:
# Para saber si llevamos una acumulado positivo de predicciones vs previsiones sobre la producción real,
# calculamos las diferencias absolutas

# Diferencia absoluta entre la predición y la producción real

df_predicciones_totales['diff_pred_real'] = abs(df_predicciones_totales['Prediccion_E_SIMEL'] - df_predicciones_totales['E_SIMEL'])

# Diferencia absoluta entre la previsión y la producción real

df_predicciones_totales['diff_prev_real'] = abs(df_predicciones_totales['PREVISION'] - df_predicciones_totales['E_SIMEL'])


# sumamos todos los valores de las columnas que queremo comparar

suma_e_simel = df_predicciones_totales['E_SIMEL'].sum()
sumas_totales_predicciones = df_predicciones_totales['Prediccion_E_SIMEL'].sum()
sumas_previsiones = df_predicciones_totales['PREVISION'].sum()


# Calculamos las diferencias entre la prediccion y la previsión respecto la producción real E_SIMEL

diferencia_prediccion_vs_produccion_real = abs(sumas_totales_predicciones - suma_e_simel)
diferencia_prevision_vs_produccion_real = abs(sumas_previsiones - suma_e_simel)


# Imprimimos los resultados para poder visualizar si mejoramos las previsiones a lo largo de todas las predicciones.

print(f"Suma de los valores en la columna E_SIMEL: {suma_e_simel}")
print(f"Suma de las predicciones: {sumas_totales_predicciones}")
print(f"Suma de las previsiones : {sumas_previsiones}")


print(f"Diferencia entre predicciones totales y E_SIMEL total: {diferencia_prediccion_vs_produccion_real}")
print(f"Diferencia entre previsiones y E_SIMEL total: {diferencia_prevision_vs_produccion_real}")


# Calculamos la diferencia entre la predicción y la previsión para saber si el modelo de predicción mejora la previsión

diferencia = diferencia_prediccion_vs_produccion_real - diferencia_prevision_vs_produccion_real

if diferencia_prediccion_vs_produccion_real > diferencia_prevision_vs_produccion_real:
    print(f"No mejoramos la predicción respecto la PREVISION real en: {diferencia}, por lo tanto, con este modelo, no estamos mejorando las previsiones.")
else:
    print(f"La predicción es MEJOR que la previsión en: {-diferencia} unidades, por lo tanto, cumplimos nuestro objetivo de mejorar la PREVISIÓN.")

Suma de los valores en la columna E_SIMEL: 622.953
Suma de las predicciones: 785.9031648635864
Suma de las previsiones : 696.6
Diferencia entre predicciones totales y E_SIMEL total: 162.95016486358645
Diferencia entre previsiones y E_SIMEL total: 73.64700000000005
No mejoramos la predicción respecto la PREVISION real en: 89.3031648635864, por lo tanto, con este modelo, no estamos mejorando las previsiones.


In [107]:
suma_prevision = df_final['PREVISION'].sum()
suma_prevision

696.5999999999999

In [108]:
suma_E_SIMEL = df_final['E_SIMEL'].sum()
suma_E_SIMEL

622.953